In [41]:
%matplotlib notebook

import os
import numpy as np
import matplotlib.pyplot as plt

# plt.rcParams['pgf.texsystem'] = 'pdflatex'
# plt.rcParams.update({'font.family': 'serif', 'font.size': 18,
#                      'figure.titlesize' : 28,
#                      'axes.labelsize': 20,'axes.titlesize': 24,
#                      'legend.fontsize': 20, 'legend.handlelength': 2})
# plt.rcParams['text.usetex'] = True
plt.rcParams["figure.figsize"] = (10, 6)

from matplotlib.colors import Normalize # colormaps
from scipy.optimize import curve_fit # curve fitting
from scipy.signal.windows import get_window # FFT windowing

from tools import datareading, rivuletfinding, utility, datasaving

This script takes a tiff raw (16 bits) video (i.e. a folder containing numerous .tiff files) and splits it into several GCV (GevCaptureVideos) files

In [42]:
# Dataset selection
dataset = 'f50_d95_110'
dataset_path = '../' + dataset
print('Available acquisitions:', datareading.find_available_videos(dataset_path))

Available acquisitions: ['f50d100', 'f50d100_0', 'f50d100_1000', 'f50d100_1500', 'f50d100_2000', 'f50d100_2400', 'f50d100_3000', 'f50d100_3500', 'f50d105', 'f50d105_0', 'f50d105_1000', 'f50d105_1500', 'f50d105_2000', 'f50d105_2400', 'f50d105_3000', 'f50d105_3500', 'f50d110', 'f50d110_0', 'f50d110_1000', 'f50d110_1500', 'f50d110_2000', 'f50d110_2400', 'f50d110_3000', 'f50d90', 'f50d90_0', 'f50d90_1000', 'f50d90_1500', 'f50d90_2000', 'f50d90_2400', 'f50d90_3000', 'f50d90_3500', 'f50d95', 'f50d95_0', 'f50d95_1000', 'f50d95_1500', 'f50d95_2000', 'f50d95_2400', 'f50d95_3000', 'f50d95_3500']


In [43]:
# Acquisition selection : select the acquisition you wish to work on
acquisition = 'f50d105'
acquisition_path = os.path.join(dataset_path, acquisition)
datareading.is_this_a_video(acquisition_path)

True

In [44]:
number_of_frames = datareading.get_number_of_available_frames(acquisition_path)

print(f'There are {number_of_frames} frames')

captureCameraName = '"Chronos 1.4"' # name of the camera
acquisition_frequency = 1000. #FPS
exposure_time = 225. #exp
captureProg = '"Chronos software version ?"'

There are 14001 frames


In [45]:
intervals = {'0':       [0,     1997],
             '1000':     [1998,  3997],
             '1500':     [3998,  5997],
             '2000':     [5998,  7997],
             '2400':     [7998,  9997],
             '3000':    [9998,  11997],
             '3500':    [11998, 13997]}
# intervals = {'0':       [0,     1997],
#              '1000':     [1998,  3997],
#              '1500':     [3998,  5997],
#              '2000':     [5998,  7997],
#              '2400':     [7998,  10257],
#              '3000':    [10258, number_of_frames-1]}

In [46]:
for i, interval_name in enumerate(intervals):
    print(f'Interval {i+1} ("{interval_name}"): frames {intervals[interval_name][0]} to {intervals[interval_name][1]}')

    framenumbers = np.arange(intervals[interval_name][0], intervals[interval_name][1] + 1)
    frames = datareading.get_frames(acquisition_path, framenumbers = framenumbers)
    length, height, width = frames.shape
    print(frames.shape, end=' ; ')
    print(frames.dtype)

    sub_acquisition = f'{acquisition}_{interval_name}'

    gcv_path = os.path.join(dataset_path, sub_acquisition + '.gcv')
    os.mkdir(gcv_path)

    sub_acquisition_path = os.path.join(gcv_path, sub_acquisition)

    # WRITE THE VIDEO IN RAW FORM
    rawvideo_path = sub_acquisition_path + '.raw'
    frames.tofile(rawvideo_path)

    # WRITE THE META
    meta:datareading.Meta = {}
    meta['usingROI'] = 'false'
    meta['subRegionX'] = '0'
    meta['subRegionY'] = '0'
    meta['subRegionWidth'] = str(width)
    meta['subRegionHeight'] = str(height)
    meta['captureCameraName'] = str(captureCameraName)
    meta['captureFrequency'] = str(acquisition_frequency)
    meta['captureExposureTime'] = str(exposure_time)
    meta['captureProg'] = str(captureProg)

    meta_path = sub_acquisition_path + '.meta'
    try:
        with open(meta_path, 'w') as meta_file:
            for i, key in enumerate(meta):
                meta_file.write(f'{key}={meta[key]}')
                if i < len(meta) - 1:
                    meta_file.write('\n')
    except:
        raise(Exception(f'ERROR: Problem with the {sub_acquisition_path} meta file (probably it could not be opened).'))

    # WRITE THE STAMPS
    ms_per_frame = 1000/acquisition_frequency
    ns_per_frame = 1e9/acquisition_frequency
    stamps:datareading.Stamps = {'framenumber': np.arange(length, dtype=int),
                                 'camera_time': (np.arange(length, dtype=float) * ns_per_frame).astype(np.int64),
                                 'computer_time': (np.arange(length, dtype=float) * ms_per_frame).astype(np.int64)}

    stamps_path = sub_acquisition_path + '.stamps'
    try:
        with open(stamps_path, 'w') as stamps_file:
            for i_line in range(length):
                stamps_file.write(f'{stamps["framenumber"][i_line]}\t{stamps["camera_time"][i_line]}\t{stamps["computer_time"][i_line]}')
                if i_line < length - 1:
                    stamps_file.write('\n')
    except:
        raise(Exception(f'ERROR: Problem with the {sub_acquisition_path} stamps file (probably it could not be opened).'))



Interval 1 ("0"): frames 0 to 1997
(1998, 360, 1280) ; uint8


FileExistsError: [Errno 17] File exists: '../f50_d95_110/f50d105_0.gcv'